# Offline monitoring

This Jupyter Notebook helps to visualize the L60 data monitoring plots.

In [ ]:
%matplotlib widget
import os, sys, json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import dates
import pickle as pkl
import ipywidgets as widget
import ipydatetime
import datetime
import numpy as np
from datetime import datetime, timezone

def get_day_hour(date):
    start = date.split('_')[0][:-1]
    stop = date.split('_')[1][:-1]
    start_day = datetime.strptime(start,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    stop_day = datetime.strptime(stop,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    
    return start_day+" -> "+stop_day

## Load data
Let's load the pickle files. Germanium detectors and SiPMs will be separately analyzed (considering different parameters are plotted for each one of them).

In [ ]:
with open("config.json") as f: 
    j_config = json.load(f)

exp         = j_config['run_info']['exp']
period      = j_config['period']
run         = j_config['run']
datatype    = j_config['datatype']

pkl_name = exp + '-' + period + '-' + run + '-' + datatype 
print("The following analysis will be inspected:", pkl_name)

pkl_files = os.listdir('out/pkl-files/par-vs-time/')

geds_list = [file for file in pkl_files if 'string' in file]      # geds: grouped by string
spms_list = [file for file in pkl_files if 'string' not in file]  # spms: everything not grouped by string

# geds
if geds_list != []:
    # configuration
    geds_map = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in geds_list]))
    # parameters
    geds_par = list(dict.fromkeys([file.split('-')[-2] for file in geds_list]))
    geds_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in geds_list])))
    # date
    print("\ngeds map:", geds_map)
    print("geds par:", geds_par)
else: 
    geds_map = geds_par = geds_date = []
    print("\n-> NO data for geds was found")

geds_date_formatted = [get_day_hour(date) for date in geds_date]
geds_time_option = [(key, value) for key,value in zip(geds_date_formatted, geds_date)]

# spms
if spms_list != []:
    # configuration
    spms_map  = list(dict.fromkeys([file.split('-')[-2].split('.')[0] for file in spms_list if 'OB' in file if 'gain' not in file]))
    spms_map  += list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list if 'IB' in file if 'gain' not in file]))  
    spms_map_gain  = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list if 'gain' in file]))  
    spms_map_div = []
    for entry in spms_map:
        if entry=='top_OB' or entry=='bot_OB':
            spms_map_div.append(entry+'-1')
            spms_map_div.append(entry+'-2')
        else:
            spms_map_div.append(entry)
    # parameters
    spms_par  = list(dict.fromkeys([file.split('-')[-3] for file in spms_list if 'OB' in file if 'gain' not in file]))
    spms_par  = list(dict.fromkeys([file.split('-')[-2] for file in spms_list if 'IB' in file if 'gain' not in file]))
    spms_par_gain  = list(dict.fromkeys([file.split('-')[-2] for file in spms_list if 'gain' in file]))
    # date
    spms_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in spms_list])))
    
    print("\nspms map (div):", spms_map_div)
    if spms_map_gain!=[]: print("spms map (gain):", spms_map_gain)
    print("spms par:", spms_par_gain+spms_par)
else:    
    spms_map = spms_map_gain = spms_map_div = spms_par = spms_par_gain = spms_date = []
    print("\n-> NO data for spms was found")   

spms_date_formatted = [get_day_hour(date) for date in spms_date]
spms_time_option = [(key, value) for key,value in zip(spms_date_formatted, spms_date)]

## Interactive widgets
Define here the interactive widgets you want to include in the final plots. Data+time selection widget still needs to be implemented.

In [ ]:
if geds_date!=[]:
    # select widget for selecting the date (for geds) 
    geds_time_select = widget.Select(
        options=geds_time_option,
        value=geds_date[0],
        # rows=10,
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
    )

if spms_date!=[]:
    # select widget for selecting the date (for spms)
    spms_time_select = widget.Select(
        options=spms_time_option,
        value=spms_date[0],
        # rows=10,
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
    )


# tab for selecting parameters to plot (for geds)
par_geds_buttons = widget.ToggleButtons(
    options= geds_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# tab for selecting parameters to plot (for spms)
par_spms_buttons = widget.ToggleButtons(
    options= spms_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# par-vs-time
geds_map_buttons = widget.ToggleButtons(
    options= geds_map,
    description='String #:',
    disabled=False,
    button_style='',
    tooltips=[],
) 
spms_map_buttons = widget.ToggleButtons(
    options= spms_map_div, 
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# gain
spms_map_buttons_gain = widget.ToggleButtons(
    options= spms_map_gain,
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# heatmaps
spms_map_buttons_heatmap = widget.ToggleButtons(
    options= ['OB' ,'IB'],
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# Germanium detectors

## par-vs-time plots

In [ ]:
# geds
def plot_geds(time_select, parameter, string):
    ax = pkl.load(open(f'out/pkl-files/par-vs-time/{pkl_name}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    plt.tight_layout()
    plt.grid()
    plt.show()
    
if geds_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()
out_geds = widget.interactive_output(plot_geds, {'time_select':geds_time_select,
                                                 'parameter'  :par_geds_buttons, 
                                                 'string'     :geds_map_buttons})

widget.VBox([widget.HBox([geds_time_select,widget.VBox([par_geds_buttons, geds_map_buttons])]),out_geds])

## heatmaps

In [ ]:
# geds
def plot_geds_heatmap(time_select, parameter):
    fig = pkl.load(open(f'out/pkl-files/heatmaps/{pkl_name}-{time_select}-{parameter}.pkl', 'rb'))
    plt.tight_layout()
    plt.show()
    
if geds_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()
out_geds_heatmap = widget.interactive_output(plot_geds_heatmap, {'time_select':geds_time_select,
                                                         'parameter'  :par_geds_buttons})

widget.VBox([widget.HBox([geds_time_select,widget.VBox([par_geds_buttons])]),out_geds_heatmap])

# SiPM detectors 

## par-vs-time plots

In [ ]:
# spms
def plot_spms(time_select, parameter, string):
    ax = pkl.load(open(f'out/pkl-files/par-vs-time/{pkl_name}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    plt.tight_layout()
    plt.grid()
    plt.show()
    
if spms_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()   
out_spms = widget.interactive_output(plot_spms, {'time_select':spms_time_select,
                                                 'parameter'  :par_spms_buttons,
                                                 'string'     :spms_map_buttons})

widget.VBox([widget.HBox([spms_time_select,widget.VBox([par_spms_buttons, spms_map_buttons])]),out_spms])

## gain plots

In [ ]:
# spms
def plot_spms_gain(time_select, string):
    ax = pkl.load(open(f'out/pkl-files/par-vs-time/{pkl_name}-{time_select}-gain-{string}.pkl', 'rb'))
    ax[0].plot()
    plt.tight_layout()
    plt.show()
    
if 'gain' not in spms_par_gain:
    print(f'There is no gain to plot!')
    sys.exit()   
out_spms_gain = widget.interactive_output(plot_spms_gain, {'time_select':spms_time_select,
                                                           'string'     :spms_map_buttons_gain})

widget.VBox([widget.HBox([spms_time_select,widget.VBox([spms_map_buttons_gain])]),out_spms_gain])

## heatmaps

In [ ]:
# spms
def plot_spms_heatmap(time_select, parameter, string):
    fig = pkl.load(open(f'out/pkl-files/heatmaps/{pkl_name}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    plt.tight_layout()
    plt.show()
    
if spms_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()   
out_spms_heatmap = widget.interactive_output(plot_spms_heatmap, {'time_select':spms_time_select,
                                                 'parameter'  :par_spms_buttons,
                                                 'string'     :spms_map_buttons_heatmap})

widget.VBox([widget.HBox([spms_time_select,widget.VBox([par_spms_buttons, spms_map_buttons_heatmap])]),out_spms_heatmap])